In [8]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
import rdkit
print(rdkit.__version__)

2022.03.5


In [27]:
import os 

ms = []
names = []

for dir in os.listdir('./data'):
    if os.path.isdir('./data/'+dir):
        for file in os.listdir('./data/'+dir):
            if file.endswith('.pdb'):
                names.append(dir)
                m = rdkit.Chem.rdmolfiles.MolFromPDBFile('./data/'+dir+"/"+file)
                ms.append(m)
                print(type(m))




<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class 'rdkit.Chem.rdchem.Mol'>
<class '

[12:55:07] WARNING: not removing hydrogen atom without neighbors
[12:55:07] Explicit valence for atom # 0 Cl, 2, is greater than permitted


In [29]:
#ms = [x for x in Chem.SmilesMolSupplier('../data/BLSets_selected_actives.txt') if x.GetProp('_Name')=='CHEMBL204']
#len(ms)

print(len(ms))

#from rdkit.Chem import Draw
#img = Draw.MolToImage(ms[0])

mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=2048)

# bit vectors:
fp = mfpgen.GetFingerprint(ms[0])
sfp = mfpgen.GetSparseFingerprint(ms[0])

# count vectors:
cfp = mfpgen.GetCountFingerprint(ms[0])
scfp = mfpgen.GetSparseCountFingerprint(ms[0])

print(f'fp: {type(fp)} {len(fp)}')
print(f'sfp: {type(sfp)} {len(sfp)}')
print(f'cfp: {type(cfp)} {cfp.GetLength()}')
print(f'scfp: {type(scfp)} {scfp.GetLength()}')

152
fp: <class 'rdkit.DataStructs.cDataStructs.ExplicitBitVect'> 2048
sfp: <class 'rdkit.DataStructs.cDataStructs.SparseBitVect'> 4294967295
cfp: <class 'rdkit.DataStructs.cDataStructs.UIntSparseIntVect'> 2048
scfp: <class 'rdkit.DataStructs.cDataStructs.ULongSparseIntVect'> 18446744073709551615
